# Getting Started with HySplit

**hysplit** is a Python package for conducting trajectory and dispersion modeling with HYSPLIT (HYbrid Single-Particle Lagrangian Integrated Trajectory). This is a Python port of the R [splitr](https://github.com/rich-iannone/splitr) package.

This tutorial covers:
1. Installation
2. Trajectory modeling
3. Dispersion modeling
4. Cluster computing workflows (download + run phases)
5. Batch processing

## 1. Installation

Install from PyPI:

In [ ]:
# Basic installation
!pip install hysplit

# With visualization support
# !pip install "hysplit[viz]"

# With all optional dependencies
# !pip install "hysplit[all]"

## 2. Trajectory Modeling

Trajectory models track the path of air masses moving forward or backward in time. This is useful for:
- Determining where air at a location came from (backward trajectory)
- Predicting where air from a source will go (forward trajectory)

In [ ]:
import hysplit
from datetime import datetime

print(f"hysplit version: {hysplit.__version__}")

### 2.1 Simple Trajectory Run

Run a 24-hour forward trajectory from Ontario, Canada:

In [ ]:
# Run trajectory model
trajectory = hysplit.hysplit_trajectory(
    lat=42.83752,              # Latitude (Ontario, Canada)
    lon=-80.30364,             # Longitude
    height=50,                 # Height above ground (meters)
    duration=24,               # Duration in hours
    days=["2012-03-12"],       # Date(s) to run
    daily_hours=[0, 6, 12, 18], # Hours to run each day
    direction="forward",       # Direction: "forward" or "backward"
    met_type="reanalysis",     # Meteorological data type
    extended_met=False,        # Include extended meteorology
    met_dir="./met",           # Directory for met files
    exec_dir="./out"           # Directory for output
)

print(f"Trajectory shape: {trajectory.shape}")
trajectory.head(10)

### 2.2 Understanding the Output

The trajectory DataFrame contains:

| Column | Description |
|--------|-------------|
| `run` | Index for individual model run |
| `receptor` | Numeric label for the 3D receptor position |
| `hour_along` | Hours from start (positive=forward, negative=backward) |
| `traj_dt` | Date-time of trajectory position |
| `lat`, `lon`, `height` | Position along trajectory |
| `pressure` | Air pressure (hPa) |

In [ ]:
# Summary statistics
print("Trajectory Summary:")
print(f"  Number of runs: {trajectory['run'].nunique()}")
print(f"  Lat range: {trajectory['lat'].min():.4f} - {trajectory['lat'].max():.4f}")
print(f"  Lon range: {trajectory['lon'].min():.4f} - {trajectory['lon'].max():.4f}")
print(f"  Height range: {trajectory['height'].min():.1f} - {trajectory['height'].max():.1f} m")

### 2.3 Method Chaining (Pipeline) API

For more complex workflows, use the object-oriented pipeline approach:

In [ ]:
# Create model using method chaining
trajectory_model = (
    hysplit.create_trajectory_model()
    .add_trajectory_params(
        lat=43.45,
        lon=-79.70,
        height=50,
        duration=6,
        days=["2015-07-01"],
        daily_hours=[0, 12],
        direction="backward",
        met_type="reanalysis",
        met_dir="./met",
        exec_dir="./out"
    )
    .run()
)

# Extract the trajectory data
trajectory_df = trajectory_model.get_output_tbl()
trajectory_df

### 2.4 Plotting Trajectories

Visualize trajectories on an interactive map:

In [ ]:
# Plot trajectory on interactive map
hysplit.trajectory_plot(trajectory_df)

### 2.5 Extended Meteorology

Get additional meteorological variables along the trajectory:

In [ ]:
# Run with extended meteorology
trajectory_ext = hysplit.hysplit_trajectory(
    lat=42.83752,
    lon=-80.30364,
    height=50,
    duration=24,
    days=["2012-03-12"],
    daily_hours=[12],
    direction="forward",
    met_type="reanalysis",
    extended_met=True,  # Enable extended meteorology
    met_dir="./met",
    exec_dir="./out"
)

# Extended columns include:
# theta, air_temp, rainfall, mixdepth, rh, sp_humidity, h2o_mixrate, terr_msl, sun_flux
print(f"Columns with extended_met=True: {list(trajectory_ext.columns)}")

## 3. Dispersion Modeling

Dispersion models simulate the transport and spread of particles or gases from emission sources.

In [ ]:
from datetime import timedelta

start_time = datetime(2015, 7, 1, 0, 0)

# Create dispersion model
dispersion_model = (
    hysplit.create_dispersion_model()
    .add_source(
        name="particle",
        lat=49.0,                    # Vancouver, Canada
        lon=-123.0,
        height=50,                   # Emission height (m)
        rate=5,                      # Emission rate (mass units/hour)
        pdiam=15,                    # Particle diameter (micrometers)
        density=1.5,                 # Particle density (g/cm³)
        shape_factor=0.8,            # Shape factor (0-1)
        release_start=start_time,
        release_end=start_time + timedelta(hours=2)
    )
    .add_dispersion_params(
        start_time=start_time,
        end_time=start_time + timedelta(hours=6),
        direction="forward",
        met_type="reanalysis",
        met_dir="./met",
        exec_dir="./out"
    )
    .run()
)

# Get dispersion output
dispersion_df = dispersion_model.get_output_tbl()
print(f"Dispersion shape: {dispersion_df.shape}")
dispersion_df.head(10)

### 3.1 Plotting Dispersion Data

In [ ]:
# Plot particle positions
dispersion_model.dispersion_plot()

## 4. Cluster Computing Workflows

For HPC environments without internet access, use the two-phase workflow:

1. **Download phase**: Download meteorological data on a machine with internet
2. **Run phase**: Execute models on the cluster using pre-downloaded data

### 4.1 Phase 1: Download Meteorological Data

Run this on a machine with internet access:

In [ ]:
from hysplit.workflows import download_met_data, create_met_manifest

# Download all needed meteorological files for a date range
manifest = download_met_data(
    met_type="reanalysis",
    start_date="2012-03-01",
    end_date="2012-03-31",
    output_dir="/data/met/reanalysis",
    buffer_days=2,             # Extra days for boundary conditions
    compute_checksums=True,    # For data validation
    n_workers=4                # Parallel downloads
)

print(f"Downloaded {len(manifest['files'])} files")
print(f"Total size: {manifest['total_size_mb']:.1f} MB")

# Save manifest for transfer to cluster
create_met_manifest(manifest, "/data/met/manifest.json")
print("Manifest saved to /data/met/manifest.json")

### 4.2 Phase 2: Run Models Offline

Run this on the cluster (no internet required):

In [ ]:
from hysplit.workflows import run_trajectory_offline, load_met_manifest

# Load pre-downloaded manifest
manifest = load_met_manifest("/scratch/met/manifest.json")

# Validate data integrity (optional but recommended)
from hysplit.workflows import validate_met_data
validation = validate_met_data("/scratch/met/manifest.json", check_checksums=True)
print(f"Validation: {validation['status']}")

# Run trajectory without internet
trajectory = run_trajectory_offline(
    lat=42.83752,
    lon=-80.30364,
    height=50,
    duration=24,
    days=["2012-03-12"],
    met_manifest=manifest,
    exec_dir="/scratch/output"
)

trajectory.head()

## 5. Batch Processing

Run many trajectories in parallel for maximum performance:

In [ ]:
from hysplit.workflows import create_batch_config, run_batch_trajectories

# Define batch configuration
config = create_batch_config(
    locations=[
        {"lat": 42.83752, "lon": -80.30364},  # Ontario
        {"lat": 43.65107, "lon": -79.34702},  # Toronto
        {"lat": 45.50169, "lon": -73.56725},  # Montreal
        {"lat": 49.28273, "lon": -123.12074}, # Vancouver
        {"lat": 51.04532, "lon": -114.05719}, # Calgary
    ],
    days=["2012-03-10", "2012-03-11", "2012-03-12"],
    daily_hours=[0, 6, 12, 18],
    duration=48,
    met_dir="/data/met",
    exec_dir="/scratch/output"
)

print(f"Batch configuration:")
print(f"  Total runs: {config.total_runs}")
print(f"  Locations: {len(config.locations)}")
print(f"  Days: {len(config.days)}")
print(f"  Hours per day: {len(config.daily_hours)}")

In [ ]:
# Run with 8 parallel workers
results = run_batch_trajectories(config, n_workers=8)

print(f"Completed {len(results)} trajectory runs")
print(f"Total rows: {sum(len(df) for df in results.values())}")

### 5.1 SLURM Job Array Generation

For cluster environments with SLURM, generate job array scripts:

In [ ]:
# Generate SLURM job array script
slurm_script = config.to_slurm_array(
    script_path="run_trajectories.sh",
    python_script="run_single_trajectory.py",
    partition="compute",
    time="01:00:00",
    memory="4G"
)

print("SLURM script generated:")
print(slurm_script[:500])

## 6. Meteorological Data Types

Available meteorological data sources:

In [ ]:
import pandas as pd

met_types = pd.DataFrame({
    "Type": ["gdas1", "gdas0p5", "reanalysis", "narr", "nam12", "gfs0p25", "era5"],
    "Description": [
        "GDAS 1-degree",
        "GDAS 0.5-degree",
        "NCEP/NCAR Reanalysis",
        "North American Regional Reanalysis",
        "NAM 12 km",
        "GFS 0.25-degree",
        "ERA5 Reanalysis"
    ],
    "Resolution": ["1°", "0.5°", "2.5°", "32 km", "12 km", "0.25°", "0.25°"],
    "Coverage": ["Global", "Global", "Global", "N. America", "N. America", "Global", "Global"]
})

met_types

## 7. Configuration Options

Customize HYSPLIT parameters:

In [ ]:
# Create custom configuration
config = hysplit.set_config(
    numpar=5000,       # Number of particles
    maxpar=20000,      # Maximum particles
    tratio=0.75,       # Time step ratio
    delt=0.0,          # Integration time step (0=auto)
    mgmin=10,          # Minimum mixing depth
    kmixd=0,           # Mixing depth option
    kmix0=250,         # Minimum mixing depth for unstable
    kzmix=0,           # Vertical mixing option
    kdef=0,            # Deformation option
    kbls=1,            # Boundary layer scheme
    kblt=2,            # Turbulence option
    extended_met=True  # Include extended meteorology
)

print("Configuration parameters:")
for key, value in config.__dict__.items():
    if not key.startswith('_'):
        print(f"  {key}: {value}")

## 8. Data Analysis with pandas

The output DataFrames can be analyzed with pandas:

In [ ]:
# Example analysis: Group by run and compute statistics
trajectory_stats = trajectory.groupby('run').agg({
    'lat': ['mean', 'min', 'max'],
    'lon': ['mean', 'min', 'max'],
    'height': ['mean', 'min', 'max'],
    'pressure': ['mean', 'min', 'max']
}).round(2)

trajectory_stats

In [ ]:
# Filter trajectories above certain height
high_altitude = trajectory[trajectory['height'] > 500]
print(f"Points above 500m: {len(high_altitude)} ({len(high_altitude)/len(trajectory)*100:.1f}%)")

## 9. Visualization with Matplotlib

Create static plots for publications:

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Trajectory paths
ax = axes[0]
for run_id in trajectory['run'].unique():
    run_data = trajectory[trajectory['run'] == run_id]
    ax.plot(run_data['lon'], run_data['lat'], 'o-', markersize=3, 
            label=f'Run {run_id}', alpha=0.7)

ax.scatter(trajectory['lon'].iloc[0], trajectory['lat'].iloc[0], 
           c='red', s=100, marker='*', zorder=10, label='Start')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Trajectory Paths')
ax.legend(loc='best', fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Height profile
ax = axes[1]
for run_id in trajectory['run'].unique():
    run_data = trajectory[trajectory['run'] == run_id]
    ax.plot(run_data['hour_along'], run_data['height'], 'o-', 
            markersize=3, label=f'Run {run_id}', alpha=0.7)

ax.set_xlabel('Hours Along Trajectory')
ax.set_ylabel('Height (m)')
ax.set_title('Height Profile')
ax.legend(loc='best', fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('trajectory_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

This tutorial covered:

1. **Installation** - `pip install hysplit` or `pip install "hysplit[all]"`
2. **Trajectory modeling** - `hysplit_trajectory()` or `create_trajectory_model()`
3. **Dispersion modeling** - `create_dispersion_model()` with `add_source()`
4. **Cluster workflows** - `download_met_data()` + `run_trajectory_offline()`
5. **Batch processing** - `run_batch_trajectories()` with parallel workers

For more information:
- [GitHub Repository](https://github.com/quishpi/hysplit)
- [HYSPLIT Documentation](https://www.ready.noaa.gov/HYSPLIT.php)
- [Original R splitr Package](https://github.com/rich-iannone/splitr)